# Summary

### Read and Inspect Data
* Fix column names to be of standard format column_name

### Fix Data
* Are all numeric columns floats?
* Do all local authorities have data for 7 years?

### Fill Missing Data
* drop local authority which does not have data for any year e.g. "Isles of Scilly"
* for some local authorities, one year of data is missing. Fill that as mean of other years.
* For some local authorities, some fields are missing. Fill it using already available total incidents data. Fix missing fields so that total remains valid. Lets choose default columns to adjust in case total!=sum(cols)
 * highway
 * primary_waste_type_measures_other_unidentified
 * single_black_bag
* For some local authorities, total_clearance_cost!=sum(individual_cost)
 * On analysing further we found that this data is very unreliable
 * 75% of the data falls under standard cost for ['single_black_bag','single_item', 'car_boot_or_less', 'small_van_load', 'transit_van_load']
 * use this data to recalculate individual costs
 * then recalculate total cost.
 * As the total and total_clearnace_cost in raw data does not deviate more than 1% from our recalculated data, we are going to drop the raw data
 
### Write checked, cleaned, filled data into a new file

In [1]:
%run import_util.py
import scan_api
import util

START loading util functions
DONE loading util functions


In [2]:
util.data_directory="./data/"
print(util.data_directory)

./data/


In [3]:
filename='flyTipping.csv'
n=5
# util.debug=True

# Read and Inspect Data
* Fix column names to be of standard format column_name

In [4]:
df = util.read_file_extraParams(filename, skiprows=range(0, 1))

In [5]:
df.head(n)

,Year,ONS Code,LA Name,Region,Total Incidents,Highway Incidents,Footpath / Bridleway Incidents,Back Alleyway Incidents,Railway Incidents,Council Land Incidents,...,Car Boot or Less Clearance Costs (£),Small Van Load Incidents,Small Van Load Clearance Costs (£),Transit Van Load Incidents,Transit Van Load Clearance Costs (£),Tipper Lorry Load Incidents,Tipper Lorry Load Clearance Costs (£),Significant / Multi Loads Incidents,Sig / Multi Loads Clearance Costs (£),Total Incidents Clearance Costs (£)
0,2012-13,*Total,*Total,*Total England,714637.0,324284.0,96009.0,114044.0,349.0,125968.0,...,6078248.0,222545.0,12462494.0,76569.0,8805444.0,11501.0,3344394,5202.0,1784224,36585739.0
1,2012-13,*Total,*Total,East Midlands,40059.0,20345.0,4808.0,1017.0,29.0,8717.0,...,373578.0,13336.0,746816.0,3452.0,396980.0,400.0,136717,121.0,44975,1930602.0
2,2012-13,*Total,*Total,East,56484.0,20857.0,3737.0,8732.0,21.0,19750.0,...,428040.0,22550.0,1262800.0,6268.0,720820.0,805.0,212501,251.0,126105,3032202.0
3,2012-13,*Total,*Total,London,231894.0,150896.0,44594.0,5502.0,64.0,14904.0,...,2316404.0,63764.0,3570784.0,19204.0,2208460.0,2000.0,680498,1193.0,438207,10803983.0
4,2012-13,*Total,*Total,North East,56748.0,5297.0,6653.0,25961.0,53.0,15435.0,...,466146.0,21246.0,1189776.0,6738.0,774870.0,1086.0,327152,358.0,122710,3136636.0


In [6]:
#replace special chars in col names, and make them standard format of col_name
def replace_special_chars_colNames(x):
    return x.lower().replace(' (£)','').replace(' -','').replace(' incidents','') \
        .replace('(','').replace(')','') \
        .replace(' ','_').replace('/','or').lower()
    
#normalise column names
def normalise_colNames(df):
    df.columns=list(map(replace_special_chars_colNames,df.columns))
    df.rename(columns={'sig_or_multi_loads_clearance_costs':'significant_or_multi_loads_clearance_costs'},inplace=True)

    
normalise_colNames(df)
df.head(5)

,year,ons_code,la_name,region,total,highway,footpath_or_bridleway,back_alleyway,railway,council_land,...,car_boot_or_less_clearance_costs,small_van_load,small_van_load_clearance_costs,transit_van_load,transit_van_load_clearance_costs,tipper_lorry_load,tipper_lorry_load_clearance_costs,significant_or_multi_loads,significant_or_multi_loads_clearance_costs,total_clearance_costs
0,2012-13,*Total,*Total,*Total England,714637.0,324284.0,96009.0,114044.0,349.0,125968.0,...,6078248.0,222545.0,12462494.0,76569.0,8805444.0,11501.0,3344394,5202.0,1784224,36585739.0
1,2012-13,*Total,*Total,East Midlands,40059.0,20345.0,4808.0,1017.0,29.0,8717.0,...,373578.0,13336.0,746816.0,3452.0,396980.0,400.0,136717,121.0,44975,1930602.0
2,2012-13,*Total,*Total,East,56484.0,20857.0,3737.0,8732.0,21.0,19750.0,...,428040.0,22550.0,1262800.0,6268.0,720820.0,805.0,212501,251.0,126105,3032202.0
3,2012-13,*Total,*Total,London,231894.0,150896.0,44594.0,5502.0,64.0,14904.0,...,2316404.0,63764.0,3570784.0,19204.0,2208460.0,2000.0,680498,1193.0,438207,10803983.0
4,2012-13,*Total,*Total,North East,56748.0,5297.0,6653.0,25961.0,53.0,15435.0,...,466146.0,21246.0,1189776.0,6738.0,774870.0,1086.0,327152,358.0,122710,3136636.0


In [7]:
util.log(df.shape)
df.describe()

(2352, 45)


,total,highway,footpath_or_bridleway,back_alleyway,railway,council_land,agricultural,private_or_residential,commercial_or_industrial,watercourse_or_bank,...,single_item_clearance_costs,car_boot_or_less,car_boot_or_less_clearance_costs,small_van_load,small_van_load_clearance_costs,transit_van_load,transit_van_load_clearance_costs,tipper_lorry_load,significant_or_multi_loads,total_clearance_costs
count,2.331000e+03,2327.000000,2330.000000,2330.000000,2327.000000,2330.000000,2330.000000,2330.000000,2328.000000,2328.000000,...,1.662000e+03,2330.000000,1.662000e+03,2330.000000,1.662000e+03,2330.000000,1.662000e+03,2330.000000,2330.000000,1.644000e+03
mean,7.974410e+03,3691.519123,1296.804292,1005.424893,6.229910,1329.833476,24.666094,170.898712,57.877577,18.799399,...,3.786125e+04,2202.003863,6.276038e+04,2584.475536,1.381927e+05,1012.935622,1.142216e+05,156.915880,82.039914,4.228412e+05
std,5.498216e+04,26952.307029,9373.969312,6899.181896,62.516873,8855.464124,166.246565,1157.790556,397.761340,129.273480,...,2.539447e+05,15984.147528,4.411014e+05,17623.920099,9.255564e+05,6871.190920,7.714414e+05,1058.292923,615.733730,2.879563e+06
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
25%,5.260000e+02,184.500000,23.000000,5.000000,0.000000,35.000000,0.000000,2.000000,0.000000,0.000000,...,2.030000e+03,105.000000,2.929000e+03,159.000000,8.400000e+03,53.000000,5.635000e+03,2.000000,0.000000,2.758475e+04
50%,1.068000e+03,401.000000,71.000000,29.000000,0.000000,101.000000,1.000000,11.000000,4.000000,2.000000,...,4.698000e+03,218.000000,6.235000e+03,349.500000,1.803200e+04,134.000000,1.414500e+04,15.000000,3.000000,5.624600e+04
75%,2.845500e+03,928.500000,240.750000,190.750000,1.000000,370.750000,8.000000,42.000000,15.000000,7.000000,...,1.232500e+04,562.750000,1.619650e+04,950.750000,5.138000e+04,380.500000,4.117000e+04,52.000000,16.000000,1.624328e+05
max,1.072431e+06,496435.000000,187028.000000,118826.000000,1599.000000,165678.000000,3395.000000,24314.000000,9395.000000,3055.000000,...,5.082665e+06,307843.000000,7.956856e+06,346938.000000,1.850154e+07,140765.000000,1.618803e+07,21393.000000,14949.000000,5.796687e+07


# Fix Data
* Are all numeric columns floats?
* Do all local authorities have data for 7 years?

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 45 columns):
year                                              2352 non-null object
ons_code                                          2352 non-null object
la_name                                           2352 non-null object
region                                            2352 non-null object
total                                             2331 non-null float64
highway                                           2327 non-null float64
footpath_or_bridleway                             2330 non-null float64
back_alleyway                                     2330 non-null float64
railway                                           2327 non-null float64
council_land                                      2330 non-null float64
agricultural                                      2330 non-null float64
private_or_residential                            2330 non-null float64
commercial_or_industrial         

In [9]:
#check if all la have data for all 7 years
def check_7years_data_for_each_la(df):
    x=df.groupby('la_name').size().to_frame('size').reset_index()
    y=x[x['size']!=7]
    display(y.head(n))
        
check_7years_data_for_each_la(df)

,la_name,size
0,*Total,70
1,Adur,6
2,Adur,1
3,Allerdale,6
4,Allerdale,1


In [10]:
#fix col type
def fix_column_type(df):
    for x in df.iloc[:,4:].columns:
        if(df[x].dtypes=='object'):
            util.log(x)
            df[x]=df[x].str.replace('£','').str.replace(',','')
            df[x]=df[x].astype('float64')


#fix trailing spaces in names
def fix_trailing_spaces(df):
    for x in df.iloc[:,0:4].columns:
        df[x]=df[x].str.strip()
        
#fix la names
def fix_la_name(df):
    fixedNames={'Bristol, City of':'Bristol',
            'Shepway':'Folkestone and Hythe',
            'Herefordshire, County of':'Herefordshire',
            'Kings Lynn and West Norfolk':'King\'s Lynn and West Norfolk',
            'Newcastle-upon-Tyne':'Newcastle upon Tyne',
            'Rutland County':'Rutland',
            'St. Helens':'St Helens'
           }
    df['la_name'].replace(fixedNames,inplace=True)

fix_column_type(df)        
fix_trailing_spaces(df)
fix_la_name(df)

tipper_lorry_load_clearance_costs
significant_or_multi_loads_clearance_costs


In [11]:
#recheck all columns are float
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 45 columns):
year                                              2352 non-null object
ons_code                                          2352 non-null object
la_name                                           2352 non-null object
region                                            2352 non-null object
total                                             2331 non-null float64
highway                                           2327 non-null float64
footpath_or_bridleway                             2330 non-null float64
back_alleyway                                     2330 non-null float64
railway                                           2327 non-null float64
council_land                                      2330 non-null float64
agricultural                                      2330 non-null float64
private_or_residential                            2330 non-null float64
commercial_or_industrial         

In [12]:
#recheck if all la have data for all 7 years
check_7years_data_for_each_la(df)

,la_name,size
0,*Total,70


# Fill Missing Data
* drop local authority which does not have data for any year e.g. "Isles of Scilly"
* for some local authorities, one year of data is missing. Fill that as mean of other years.
* For some local authorities, some fields are missing. Fill it using already avialavle total incidents data. 
* For some local authorities, total_clearance_cost!=sum(individual_cost)
 * On analysing further we found that this data is very unreliable
 * 75% of the data falls under standard cost for ['single_black_bag','single_item', 'car_boot_or_less', 'small_van_load', 'transit_van_load']
 * use this data to recalculate individual costs
 * then recalculate total cost.
           


In [13]:
df.isnull().sum()

year                                                0
ons_code                                            0
la_name                                             0
region                                              0
total                                              21
highway                                            25
footpath_or_bridleway                              22
back_alleyway                                      22
railway                                            25
council_land                                       22
agricultural                                       22
private_or_residential                             22
commercial_or_industrial                           24
watercourse_or_bank                                24
other_unidentified                                 22
animal_carcass                                     23
green                                              22
vehicle_parts                                      22
white_goods                 

In [14]:
#check which rows have total<=0
display(df[df['total']<=0])

#drop  as no data avialable for any year
util.log("Shape of df, BEFORE dropping empty local authorities"+str(df.shape))
df.drop(df[df['la_name'] == 'Isles of Scilly'].index,inplace=True)
util.log("Shape of df, AFTER dropping empty local authorities"+str(df.shape))

,year,ons_code,la_name,region,total,highway,footpath_or_bridleway,back_alleyway,railway,council_land,...,car_boot_or_less_clearance_costs,small_van_load,small_van_load_clearance_costs,transit_van_load,transit_van_load_clearance_costs,tipper_lorry_load,tipper_lorry_load_clearance_costs,significant_or_multi_loads,significant_or_multi_loads_clearance_costs,total_clearance_costs
147,2012-13,E06000053,Isles of Scilly,South West,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
483,2013-14,E06000053,Isles of Scilly,South West,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
819,2014-15,E06000053,Isles of Scilly,South West,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1155,2015-16,E06000053,Isles of Scilly,South West,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1491,2016-17,E06000053,Isles of Scilly,South West,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1827,2017-18,E06000053,Isles of Scilly,South West,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN
2164,2018-19,E06000053,Isles of Scilly,South West,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN


Shape of df, BEFORE dropping empty local authorities(2352, 45)
Shape of df, AFTER dropping empty local authorities(2345, 45)


In [15]:
#check la which have missing year
def check_for_la_with_missing_years(df):
    df1=df[df['total'].isna()]
    util.log("local authorities with missing years : " + str(df1.shape))
    display(df1.head(n))

check_for_la_with_missing_years(df)

local authorities with missing years : (21, 45)


,year,ons_code,la_name,region,total,highway,footpath_or_bridleway,back_alleyway,railway,council_land,...,car_boot_or_less_clearance_costs,small_van_load,small_van_load_clearance_costs,transit_van_load,transit_van_load_clearance_costs,tipper_lorry_load,tipper_lorry_load_clearance_costs,significant_or_multi_loads,significant_or_multi_loads_clearance_costs,total_clearance_costs
201,2012-13,E06000018,Nottingham,East Midlands,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
537,2013-14,E06000018,Nottingham,East Midlands,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873,2014-15,E06000018,Nottingham,East Midlands,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
881,2014-15,E06000029,Poole,South West,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1050,2015-16,E06000043,Brighton and Hove,South East,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#fill cols based on other years
def fill_la_with_missing_years(df):
    df1=df[df['total'].isna()]
    la_names=set(df1['la_name'])
    cols=df1.columns[4:]

    for la_name in la_names:
        row=df[df['la_name']==la_name].mean().round()

        years=list(df1[df1['la_name']==la_name]['year'])
        null_release_mask = (df1.la_name==la_name) & (df.year.isin(years))
        df1.loc[null_release_mask, cols] = list(row[cols])

    util.log("Fixed local authorities with missing years : " + str(df1.shape))
    display(df1.head(n))
    return df1

df1=fill_la_with_missing_years(df)
df=util.copyToDF(df,df1,df1.columns[4:])

Fixed local authorities with missing years : (21, 45)


,year,ons_code,la_name,region,total,highway,footpath_or_bridleway,back_alleyway,railway,council_land,...,car_boot_or_less_clearance_costs,small_van_load,small_van_load_clearance_costs,transit_van_load,transit_van_load_clearance_costs,tipper_lorry_load,tipper_lorry_load_clearance_costs,significant_or_multi_loads,significant_or_multi_loads_clearance_costs,total_clearance_costs
201,2012-13,E06000018,Nottingham,East Midlands,7619.0,5946.0,1117.0,160.0,0.0,146.0,...,NaN,914.0,NaN,454.0,NaN,127.0,19685.0,114.0,39900.0,NaN
537,2013-14,E06000018,Nottingham,East Midlands,7619.0,5946.0,1117.0,160.0,0.0,146.0,...,NaN,914.0,NaN,454.0,NaN,127.0,19685.0,114.0,39900.0,NaN
873,2014-15,E06000018,Nottingham,East Midlands,7619.0,5946.0,1117.0,160.0,0.0,146.0,...,NaN,914.0,NaN,454.0,NaN,127.0,19685.0,114.0,39900.0,NaN
881,2014-15,E06000029,Poole,South West,618.0,334.0,74.0,20.0,0.0,186.0,...,7125.0,187.0,10653.0,102.0,10508.0,4.0,1304.0,0.0,46.0,32718.0
1050,2015-16,E06000043,Brighton and Hove,South East,2528.0,1570.0,636.0,58.0,0.0,44.0,...,16508.0,523.0,28462.0,301.0,41342.0,0.0,0.0,0.0,180.0,115100.0


In [17]:
check_for_la_with_missing_years(df)

local authorities with missing years : (0, 45)


,year,ons_code,la_name,region,total,highway,footpath_or_bridleway,back_alleyway,railway,council_land,...,car_boot_or_less_clearance_costs,small_van_load,small_van_load_clearance_costs,transit_van_load,transit_van_load_clearance_costs,tipper_lorry_load,tipper_lorry_load_clearance_costs,significant_or_multi_loads,significant_or_multi_loads_clearance_costs,total_clearance_costs


In [18]:
#Now all total fields are filled
df.isnull().sum()

year                                                0
ons_code                                            0
la_name                                             0
region                                              0
total                                               0
highway                                             4
footpath_or_bridleway                               1
back_alleyway                                       1
railway                                             4
council_land                                        1
agricultural                                        1
private_or_residential                              1
commercial_or_industrial                            3
watercourse_or_bank                                 3
other_unidentified                                  1
animal_carcass                                      2
green                                               1
vehicle_parts                                       1
white_goods                 

# Fix missing fields so that total remains valid. Lets choose default columns to adjust in case total!=sum(cols)

* highway
* primary_waste_type_measures_other_unidentified
* single_black_bag


In [19]:
def check_rows_with_missing_data(df):
    df1=df.copy()
    df2=df1.dropna(how='any',subset=df1.columns[4:31])
    x=pd.concat([df1,df2]).drop_duplicates(keep=False)

    util.log("rows with missing data " + str(x.shape[0]))
    display(x.head(n))

check_rows_with_missing_data(df)

rows with missing data 9


,year,ons_code,la_name,region,total,highway,footpath_or_bridleway,back_alleyway,railway,council_land,...,car_boot_or_less_clearance_costs,small_van_load,small_van_load_clearance_costs,transit_van_load,transit_van_load_clearance_costs,tipper_lorry_load,tipper_lorry_load_clearance_costs,significant_or_multi_loads,significant_or_multi_loads_clearance_costs,total_clearance_costs
855,2014-15,E08000021,Newcastle upon Tyne,North East,8057.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1100,2015-16,E07000040,East Devon,South West,453.0,350.0,29.0,20.0,NaN,46.0,...,4321.0,170.0,9520.0,50.0,5750.0,7.0,1380.0,1.0,460.0,23635.0
1183,2015-16,E07000042,Mid Devon,South West,340.0,NaN,48.0,5.0,0.0,49.0,...,2581.0,109.0,6104.0,41.0,4715.0,4.0,350.0,4.0,NaN,14767.0
1187,2015-16,E06000042,Milton Keynes,South East,4282.0,17.0,116.0,66.0,2.0,4061.0,...,20851.0,1609.0,90104.0,1332.0,153180.0,115.0,34500.0,23.0,8140.0,318809.0
1191,2015-16,E08000021,Newcastle upon Tyne,North East,5867.0,199.0,1092.0,2061.0,5.0,2417.0,...,80388.0,1536.0,86016.0,520.0,59800.0,53.0,9600.0,38.0,7000.0,253276.0


In [20]:
def fill_rows_with_missing_data_helper(df_adjustColName):
    df,adjustColName=df_adjustColName
    x=df
    x['missing_incidents']=x['total']-x.iloc[:,5:].sum(axis=1)
    y=x[x['missing_incidents']!=0]
    util.log_debug(len(y))
    #fix the col by adding to adjustColName
    y.fillna(0,inplace=True)
    y[adjustColName]=y[adjustColName]+y['missing_incidents']
    x.drop('missing_incidents',axis=1,inplace=True) 
    y.drop('missing_incidents',axis=1,inplace=True) 
    return y
    
def fill_rows_with_missing_data(df):
    #split df in categories
    df_main=df[df['ons_code']!='*Total']
    df_loc=df_main.iloc[:,[0,1,2,3,4]+list(range(5,15))]
    df_type=df_main.iloc[:,[0,1,2,3,4]+list(range(15,30))]
    df_cost_type=df_main.iloc[:,[0,1,2,3,4]+list(range(30,44,2))]

    df_name=[(df_loc,'highway'),
             (df_type,'primary_waste_type_measures_other_unidentified'),
            (df_cost_type,'single_black_bag')]
    
    #fill the data
    for df_adjustColName in df_name:
        y=fill_rows_with_missing_data_helper(df_adjustColName)
        df=util.copyToDF(df,y,y.columns[4:])
        
    #now fill all null/Nan with 0 
    #(only costs remain to be recalculated)
    df.fillna(0,inplace=True)
    return df
        
df=fill_rows_with_missing_data(df)      

In [21]:
check_rows_with_missing_data(df)

rows with missing data 0


,year,ons_code,la_name,region,total,highway,footpath_or_bridleway,back_alleyway,railway,council_land,...,car_boot_or_less_clearance_costs,small_van_load,small_van_load_clearance_costs,transit_van_load,transit_van_load_clearance_costs,tipper_lorry_load,tipper_lorry_load_clearance_costs,significant_or_multi_loads,significant_or_multi_loads_clearance_costs,total_clearance_costs


In [22]:
#check total cost deviation
#answer: the data looks incorrect, so best to recalculate and overwrite it
def check_cost_deviation(df):
    costs=['single_black_bag','single_item', 'car_boot_or_less', 'small_van_load', 
       'transit_van_load','tipper_lorry_load', 'significant_or_multi_loads']
    costs=[x+'_clearance_costs' for x in costs]

    df1=df.copy()
    df1['missing_incidents']=df1['total_clearance_costs']-df1[costs].sum(axis=1)
    df1['missing_incidents_perc']=df1['missing_incidents']/df1['total_clearance_costs'] *100
    df1=df1[df1['missing_incidents_perc']>0]
    df1.sort_values('missing_incidents_perc', ascending=False,inplace=True)
    display(df1.shape)
    display(df1[['missing_incidents_perc']].head(n*2))
    
check_cost_deviation(df)

(30, 47)

,missing_incidents_perc
1037,27.783402
1008,11.198213
702,8.184032
93,7.956957
1202,7.360151
1188,6.813027
1029,6.434654
1040,5.362876
1028,2.598408
1032,2.498508


In [23]:
#calculate average cost of removal
def analyse_avg_cost_of_removal(df):
    costs=['single_black_bag','single_item', 'car_boot_or_less', 'small_van_load', 
           'transit_van_load','tipper_lorry_load', 'significant_or_multi_loads']

    df1=df.copy()
    df1.dropna(inplace=True)

    for x in costs:
        df1[x+'_avg_cost']= df1[x+'_clearance_costs']/df1[x]

    display(df1.iloc[:,45:].describe())

analyse_avg_cost_of_removal(df)

,single_black_bag_avg_cost,single_item_avg_cost,car_boot_or_less_avg_cost,small_van_load_avg_cost,transit_van_load_avg_cost,tipper_lorry_load_avg_cost,significant_or_multi_loads_avg_cost
count,2177.000000,2304.000000,2338.000000,2341.000000,2327.000000,1930.000000,1523.000000
mean,inf,20.713136,20.686228,39.961722,81.944239,inf,inf
std,NaN,13.101611,13.120421,25.313121,52.028080,NaN,NaN
min,-31.464883,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,160.258712,230.000000
50%,6.562500,29.000000,29.000000,56.000000,115.000000,294.798749,360.000000
75%,7.000000,29.000000,29.000000,56.000000,115.000000,350.000000,362.375728
max,inf,34.878378,36.042553,63.428571,137.348837,inf,inf


In [24]:
#apply average cost of removel
def apply_average_cost_of_removal(df):
    costs=['single_black_bag','single_item', 'car_boot_or_less', 'small_van_load', 
           'transit_van_load']
    avg_costs=[7,29,29,56,115]

    for i in range(len(costs)):
        x=costs[i]
        y=avg_costs[i]
        df[x+'_clearance_costs']= df[x]*y

    df.head(n)
    
apply_average_cost_of_removal(df)

In [25]:
check_cost_deviation(df)

(174, 47)

,missing_incidents_perc
1037,27.783402
142,25.522757
1287,8.551210
702,8.184032
93,7.956957
1029,6.434654
1050,5.576890
1040,5.362876
466,4.180126
970,3.544923


In [26]:
#recalculate total cost
def reset_total_cost(df):
    costs=['single_black_bag','single_item', 'car_boot_or_less', 'small_van_load', 
       'transit_van_load','tipper_lorry_load', 'significant_or_multi_loads']
    costs=[x+'_clearance_costs' for x in costs]

    df['total_clearance_costs']=df[costs].sum(axis=1)
    df.shape
    
reset_total_cost(df)

In [27]:
check_cost_deviation(df)

(0, 47)

,missing_incidents_perc


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2345 entries, 0 to 2351
Data columns (total 45 columns):
year                                              2345 non-null object
ons_code                                          2345 non-null object
la_name                                           2345 non-null object
region                                            2345 non-null object
total                                             2345 non-null float64
highway                                           2345 non-null float64
footpath_or_bridleway                             2345 non-null float64
back_alleyway                                     2345 non-null float64
railway                                           2345 non-null float64
council_land                                      2345 non-null float64
agricultural                                      2345 non-null float64
private_or_residential                            2345 non-null float64
commercial_or_industrial         

### As the total and total_clearnace_cost in raw data does not deviate more than 1% from our recalculated data, we are going to drop the raw data

In [29]:
def check_Total_deviation(df):
    df_total=df[ (df['ons_code']=='*Total') & (df['region']!='*Total England')][['year','total','total_clearance_costs']]
#     display(df_total)
    df_total=df_total.groupby(['year']).sum()
    
    df_main=df[df['ons_code']!='*Total'][['year','total','total_clearance_costs']]
    df_main=df_main.groupby(['year']).sum()
    
    df_total=df_total.join(df_main,rsuffix="_recalculated")
    df_total['incident_diff_perc']=(df_total['total_recalculated']-df_total['total'])/df_total['total']*100
    df_total['cost_diff_perc']=(df_total['total_clearance_costs_recalculated']-df_total['total_clearance_costs'])/df_total['total_clearance_costs']*100
    display(df_total)
    
check_Total_deviation(df)

,total,total_clearance_costs,total_recalculated,total_clearance_costs_recalculated,incident_diff_perc,cost_diff_perc
year,,,,,,
2012-13,706239.0,36191607.0,713858.0,36399128.0,1.078813,0.573395
2013-14,847856.0,45017032.0,855475.0,45374148.0,0.898620,0.793291
2014-15,897363.0,49419462.0,905600.0,49837422.0,0.917912,0.845740
2015-16,834942.0,44109761.0,893270.0,47600609.0,6.985874,7.914003
2016-17,932499.0,53972708.0,951493.0,54868889.0,2.036892,1.660434
2017-18,881669.0,51695916.0,891133.0,52285615.0,1.073419,1.140707
2018-19,943119.0,54150604.0,953802.0,54849282.0,1.132731,1.290250


In [30]:
#final check that total la* = number of rows
# unique las
df_main=df[df['ons_code']!='*Total']
unique_localAuthorities=len(df_main['la_name'].unique())

util.log('expected rows : ' + str(unique_localAuthorities*7) + ', actual rows : ' + str(df_main.shape[0]))


expected rows : 2275, actual rows : 2275


# Write checked, cleaned, filled data into a new file

In [31]:
# write data to a file
util.write_file(df_main,'final_'+filename)
df_main.shape

(2275, 45)

In [32]:
# display
df2 = util.read_file('final_'+filename)
display(df2.shape)
display(df2.head(n))
df2.info()

(2275, 45)

,year,ons_code,la_name,region,total,highway,footpath_or_bridleway,back_alleyway,railway,council_land,...,car_boot_or_less_clearance_costs,small_van_load,small_van_load_clearance_costs,transit_van_load,transit_van_load_clearance_costs,tipper_lorry_load,tipper_lorry_load_clearance_costs,significant_or_multi_loads,significant_or_multi_loads_clearance_costs,total_clearance_costs
0,2012-13,E07000223,Adur,South East,244.0,40.0,5.0,5.0,0.0,178.0,...,2958.0,54.0,3024.0,11.0,1265.0,0.0,0.0,0.0,0.0,9370.0
1,2012-13,E07000026,Allerdale,North West,811.0,158.0,108.0,216.0,1.0,96.0,...,5046.0,428.0,23968.0,51.0,5865.0,17.0,5950.0,11.0,3940.0,47879.0
2,2012-13,E07000032,Amber Valley,East Midlands,219.0,174.0,18.0,0.0,0.0,25.0,...,580.0,59.0,3304.0,31.0,3565.0,2.0,700.0,2.0,4000.0,15172.0
3,2012-13,E07000224,Arun,South East,716.0,248.0,78.0,36.0,0.0,50.0,...,4147.0,140.0,7840.0,37.0,4255.0,12.0,3851.0,3.0,1080.0,24566.0
4,2012-13,E07000170,Ashfield,East Midlands,477.0,205.0,115.0,33.0,0.0,62.0,...,4756.0,162.0,9072.0,43.0,4945.0,0.0,10.0,0.0,0.0,21079.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2275 entries, 0 to 2274
Data columns (total 45 columns):
year                                              2275 non-null object
ons_code                                          2275 non-null object
la_name                                           2275 non-null object
region                                            2275 non-null object
total                                             2275 non-null float64
highway                                           2275 non-null float64
footpath_or_bridleway                             2275 non-null float64
back_alleyway                                     2275 non-null float64
railway                                           2275 non-null float64
council_land                                      2275 non-null float64
agricultural                                      2275 non-null float64
private_or_residential                            2275 non-null float64
commercial_or_industrial         